In [8]:
# Standard Dependencies
import cv2 
import os 
import random 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

# Tensorflow Dependencies 
import tensorflow as tf
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D,Input, Flatten

In [9]:
# End Result
# Model(input = [inputImage,verificationImage] , output = [0,1])

In [19]:
# Set GPU Growth ( For GPU Based Machines )

# Avoid OOM errors by setting the GPU Memory consuption growth

gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus : 
    tf.config.experimental.set_memory_growth(gpu, True) 

In [22]:
len(gpus) # this 0 zero length is saying that My machine doesn't have any GPu

0

In [27]:
# To create Folder Using OS

POS_PATH = os.path.join("data",'postive')
NEG_PATH = os.path.join("data",'negative')
ANC_PATH = os.path.join("data",'anchor')

os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)